# Mangrove tiles

We need create vector tiles for the Mangrove restoration and typology data provided by the client [here](https://github.com/Vizzuality/mangrove-atlas/issues/331).  
The original link to the source data is [here](https://tnc.app.box.com/s/8abiem7o8lyd9x1pvpe2yn2mdibgpcqk).  
The source data is an arcgis GDB and the data is stored in gcs bucket.  
The steps this pipe will take are:  
Pre.- script to move data from their original location to the gcs bucket.  
1.- Download raw data.  
2.- Tranform raw data to geojson with (mapshaper, geopandas, ...)  
3.- Generate the mbtile file with tippecanoe  
4.- Upload the mbtile file to gcs and mapbox as a tileset.  
5.- Comunicate to Design and FE team the tileset is ready.  

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import fiona


## 1. Download raw data
For now, working on local. Awaiting GCS permissions

# 2. Transform to geoJSON

In [2]:
gdb_file ='../../../datasets/Typology_and_Restoration_Potential/Data/MOW_Global_Mangrove_Restoration_20190411.gdb'
layer = gpd.read_file(gdb_file, driver='FileGDB', layer=0)
layer.head()

,Country,Tot_Restor,Restor_pct,Rest_Area_Loss,Area_loss_pct,Rest_Area_Dgrd,Area_dgrd_pct,Total_2016,Mean_Score,SOC,AGB,People,Fish_Score,Fish_Score_Inv,Shape_Length,Shape_Area,Loss_Driver,geometry
0,American Samoa,0.00000,0,0.00,0,0.00,0,18.74,0,0.000000,0.000000,0,109000000.0,200000.0,5.350212e+04,1.638363e+07,No Data,"MULTIPOLYGON (((-19011686.866 -1613074.241, -1..."
1,Angola,580.11556,4,671.43,5,68.92,0,13286.05,62,220274.238512,30809.263129,3800,-1.0,-1.0,1.383794e+06,2.785231e+10,NPC,"MULTIPOLYGON (((1550210.491 -1230695.594, 1549..."
2,Anguilla,0.08128,4,1.27,59,0.00,0,0.87,24,407.670000,6.524752,0,1000000.0,2000000.0,2.097797e+04,1.419573e+06,No Data,"MULTIPOLYGON (((-7025249.376 2058315.576, -702..."
3,Antigua and Barbuda,14.15571,2,20.19,2,0.00,0,886.30,22,9031.556968,1078.042000,500,8000000.0,13000000.0,1.888816e+05,1.361710e+09,No Data,"MULTIPOLYGON (((-6873186.951 1921601.093, -688..."
4,Aruba,19.38207,36,20.55,38,0.00,0,33.79,46,9816.581150,1374.874165,0,71000000.0,119000000.0,3.798229e+04,2.166772e+07,No Data,"MULTIPOLYGON (((-7781925.061 1393950.107, -778..."


In [5]:
layer.crs

<Derived Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [6]:
layer = layer.to_crs('EPSG:4326')

In [7]:
layer.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [8]:
out_path = '../../../datasets/Typology_and_Restoration_Potential/Data/MOW_Global_Mangrove_Restoration.geojson'

layer.to_file(out_path, driver="GeoJSON") 

## 3. Generate the mbtile file with tippecanoe  


In [10]:
in_file = '../../../datasets/Typology_and_Restoration_Potential/Data/MOW_Global_Mangrove_Restoration.geojson'
out_file = '../../../datasets/Typology_and_Restoration_Potential/Data/MOW_Global_Mangrove_Restoration.mbtiles'
!tippecanoe -zg -o $out_file --drop-densest-as-needed --force $in_file

For layer 0, using name "MOW_Global_Mangrove_Restoration"
108 features, 20441 bytes of geometry, 7431 bytes of separate metadata, 15373 bytes of string pool
Choosing a maxzoom of -z0 for features about 2772555 feet (845075 meters) apart
Choosing a maxzoom of -z5 for resolution of about 10086 feet (3074 meters) within features
  99.9%  5/20/13  
